In [96]:
import pandas as pd
import numpy as np
import torch
from torch import nn

In [97]:
dataset = pd.read_csv("Churn_Modelling.csv")
X = dataset.iloc[:,3:-1].values
y = dataset.iloc[:,-1].values

In [98]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

X[:,2] = le.fit_transform(X[:,2])

In [99]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],
                       remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [100]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [101]:
# All the features must be scaled in deep learning

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

In [102]:
X_train,X_test,y_train,y_test = torch.from_numpy(X_train).type(torch.float32),torch.from_numpy(X_test).type(torch.float32),torch.from_numpy(y_train).type(torch.float32),torch.from_numpy(y_test).type(torch.float32)

In [103]:
class BClass(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(in_features=12,out_features=8),
            nn.ReLU(),
            nn.Linear(in_features=8,out_features=8),
            nn.ReLU(),
            nn.Linear(in_features=8,out_features=1))
    
    def forward(self,x):
        return self.layers(x)

model = BClass()

In [104]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), 
                            lr=0.01) 

In [105]:
# Calculate accuracy (a classification metric)
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100 
    return acc

In [106]:
torch.manual_seed(42)

epochs = 1000

for epoch in range(epochs):
    model.train()

    y_logits = model(X_train).squeeze()
    loss = loss_fn(y_logits,
                   y_train) 
    y_pred = torch.round(torch.sigmoid(y_logits))
    acc = accuracy_fn(y_true=y_train, 
                      y_pred=y_pred)

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Accuracy: {acc:.2f}%")

Epoch: 0 | Loss: 0.73054, Accuracy: 22.62%
Epoch: 100 | Loss: 0.36979, Accuracy: 82.09%
Epoch: 200 | Loss: 0.33538, Accuracy: 86.40%
Epoch: 300 | Loss: 0.32809, Accuracy: 86.71%
Epoch: 400 | Loss: 0.32555, Accuracy: 86.74%
Epoch: 500 | Loss: 0.32465, Accuracy: 86.79%
Epoch: 600 | Loss: 0.32413, Accuracy: 86.75%
Epoch: 700 | Loss: 0.32333, Accuracy: 86.78%
Epoch: 800 | Loss: 0.32212, Accuracy: 86.86%
Epoch: 900 | Loss: 0.32191, Accuracy: 86.86%


In [113]:
model.eval()
with torch.inference_mode():
    y_pred_test = torch.round(torch.sigmoid(model(X_test)))
y_pred_test = y_pred_test.detach().numpy() 

In [115]:
print(np.concatenate((y_pred_test.reshape(len(y_pred_test),1),y_test.reshape(len(y_test),1)),1))

[[0. 0.]
 [0. 1.]
 [0. 0.]
 ...
 [0. 0.]
 [0. 0.]
 [0. 0.]]


In [116]:
from sklearn.metrics import confusion_matrix, accuracy_score

# Assuming you have predicted labels 'y_pred' and actual labels 'y_true'
cm = confusion_matrix(y_test, y_pred_test)
accuracy = accuracy_score(y_test, y_pred_test)

print("Confusion Matrix:")
print(cm)
print("Accuracy:", accuracy)

Confusion Matrix:
[[1510   85]
 [ 204  201]]
Accuracy: 0.8555
